In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 04:13:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("create database shellida")

DataFrame[]

In [4]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
| shellida|
+---------+



In [5]:
spark.sql("use shellida")


DataFrame[]

In [15]:
spark.sql("create table employee(empno int, ename string,sal int)").show()

23/09/25 04:29:30 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT).;
'CreateTable `spark_catalog`.`shellida`.`employee`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists


In [11]:
salesdf=spark.read.option("header","true").option("inferSchema","true").csv("/home/labuser/datasets/superstore.csv")

In [12]:
salesdf.show()

+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|   ID|        OrderID| OrderDate|  ShipDate|      ShipMode|CustomerID|    CustomerName|    Segment|         City|          State|      Country|PostalCode|Market|      Region|       ProductID|       Category|Sub-Category|         ProductName|           Sales|Quantity|Discount|    Profit|ShippingCost|OrderPriority|
+-----+---------------+----------+----------+--------------+----------+----------------+-----------+-------------+---------------+-------------+----------+------+------------+----------------+---------------+------------+--------------------+----------------+--------+--------+----------+------------+-------------+
|32298| CA-2012-124891|31/07/2012|31/07/2012|      S

In [13]:
#3 modes for writing
#overwrite

salesdf.write.mode("overwrite").saveAsTable("sales")

In [16]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
| shellida|    sales|      false|
+---------+---------+-----------+



In [20]:
spark.sql("desc extended sales").show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|          ID|      int|   null|
|     OrderID|   string|   null|
|   OrderDate|   string|   null|
|    ShipDate|   string|   null|
|    ShipMode|   string|   null|
|  CustomerID|   string|   null|
|CustomerName|   string|   null|
|     Segment|   string|   null|
|        City|   string|   null|
|       State|   string|   null|
|     Country|   string|   null|
|  PostalCode|      int|   null|
|      Market|   string|   null|
|      Region|   string|   null|
|   ProductID|   string|   null|
|    Category|   string|   null|
|Sub-Category|   string|   null|
| ProductName|   string|   null|
|       Sales|   string|   null|
|    Quantity|   string|   null|
+------------+---------+-------+
only showing top 20 rows



In [17]:
sales4tabledf= spark.read.table("sales")

In [26]:
sales4tabledf.count()

102580

In [25]:
#append
salesdf.write.mode("append").saveAsTable("sales")

In [23]:
#ignore
salesdf.write.mode("ignore").saveAsTable("sales")

In [48]:
sales_US=salesdf.select("country","state","Profit").filter("country='United States'")

In [49]:
from pyspark.sql.functions import col, sum

#col function
#transformations -> groupby, agg, cast,alias, filter

sales_US =salesdf.groupBy("Country","state").agg(sum(col("Profit").cast('int')).alias("totSales")).filter("country='United States'")

In [50]:
sales_US.show()

+-------------+--------------+--------+
|      Country|         state|totSales|
+-------------+--------------+--------+
|United States|      Virginia|   17154|
|United States|South Carolina|    1774|
|United States| West Virginia|     186|
|United States|         Maine|     417|
|United States|       Montana|    1852|
|United States|       Alabama|    4645|
|United States|       Wyoming|     100|
|United States|         Idaho|     695|
|United States|    New Jersey|    6916|
|United States|      Arkansas|    3437|
|United States|      Michigan|   21869|
|United States|      Illinois|   -9298|
|United States|          Ohio|  -14490|
|United States|      New York|   65943|
|United States|      Missouri|    6208|
|United States|   Mississippi|    2445|
|United States|      Nebraska|    1421|
|United States|     Minnesota|    9820|
|United States|    Washington|   29900|
|United States|        Kansas|     794|
+-------------+--------------+--------+
only showing top 20 rows



In [30]:
salesdf.select("Region").where("Country='India'").orderBy("region",ascending=True).show()

+------------+
|      Region|
+------------+
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
+------------+
only showing top 20 rows



In [31]:
salesdf.select("Region").where("Country='India'").orderBy("region",ascending=False).show()

+------------+
|      Region|
+------------+
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
|Central Asia|
+------------+
only showing top 20 rows



In [60]:
salesdf.select("Region").where("Country='India'").sort(salesDF.region.desc().show()

SyntaxError: incomplete input (2384465725.py, line 1)

In [33]:
salesdf.select("Region","country").sort(col("region").desc()).orderBy("country",ascending=False).filter("region='Central Asia'").distinct().show()

+------------+-----------+
|      Region|    country|
+------------+-----------+
|Central Asia|      Nepal|
|Central Asia|Afghanistan|
|Central Asia| Bangladesh|
|Central Asia|  Sri Lanka|
|Central Asia|   Pakistan|
|Central Asia|      India|
+------------+-----------+



In [34]:
spark.sql("select country,region,avg(profit) from sales group by country,region").filter("Country='India'").orderBy("region").show()

+-------+------------+--------------------+
|country|      region|         avg(profit)|
+-------+------------+--------------------+
|  India|Central Asia|0.017684887459807074|
+-------+------------+--------------------+



In [51]:
# to get tyhe average sales
from pyspark.sql.functions import col, sum, avg

sales_US =salesdf.groupBy("Country","state").agg(sum(col("Profit").cast('int')).alias("totSales"),avg(col("Profit").cast('int')).alias("AvgSales")).filter("country='United States'")

In [52]:
sales_US.show()

+-------------+--------------+--------+-------------------+
|      Country|         state|totSales|           AvgSales|
+-------------+--------------+--------+-------------------+
|United States|      Virginia|   17154|  76.58035714285714|
|United States|South Carolina|    1774|  42.23809523809524|
|United States| West Virginia|     186|               46.5|
|United States|         Maine|     417|             52.125|
|United States|       Montana|    1852| 123.46666666666667|
|United States|       Alabama|    4645|  76.14754098360656|
|United States|       Wyoming|     100|              100.0|
|United States|         Idaho|     695| 33.095238095238095|
|United States|    New Jersey|    6916|   53.6124031007752|
|United States|      Arkansas|    3437|  57.28333333333333|
|United States|      Michigan|   21869|   86.0984251968504|
|United States|      Illinois|   -9298|-18.898373983739837|
|United States|          Ohio|  -14490|-30.895522388059703|
|United States|      New York|   65943| 

In [57]:
from pyspark.sql.functions import lit
sales_US.withColumn("CountryCode",lit("US")).show()

+-------------+--------------+----------+-------------------+-----------+
|      Country|         state|TotalSales|           AvgSales|CountryCode|
+-------------+--------------+----------+-------------------+-----------+
|United States|      Virginia|     17154|  76.58035714285714|         US|
|United States|South Carolina|      1774|  42.23809523809524|         US|
|United States| West Virginia|       186|               46.5|         US|
|United States|         Maine|       417|             52.125|         US|
|United States|       Montana|      1852| 123.46666666666667|         US|
|United States|       Alabama|      4645|  76.14754098360656|         US|
|United States|       Wyoming|       100|              100.0|         US|
|United States|         Idaho|       695| 33.095238095238095|         US|
|United States|    New Jersey|      6916|   53.6124031007752|         US|
|United States|      Arkansas|      3437|  57.28333333333333|         US|
|United States|      Michigan|     218

In [54]:
sales_withcolumn = salesdf.withColumn("SalesLevel", when(col("profit")>=10000,"High").when(col("profit")<10000,"Average").otherwise("NA"))  

In [55]:
sales_withcolumn.select("country","state","profit","salesLevel").show()

+-------------+---------------+----------+----------+
|      country|          state|    profit|salesLevel|
+-------------+---------------+----------+----------+
|United States|       New York|  762.1845|   Average|
|    Australia|New South Wales|       0.1|   Average|
|    Australia|     Queensland|       0.1|   Average|
|      Germany|         Berlin|       0.1|   Average|
|      Senegal|          Dakar|         0|   Average|
|    Australia|New South Wales|       0.1|   Average|
|  New Zealand|     Wellington|         0|   Average|
|  New Zealand|        Waikato|         0|   Average|
|United States|     California|  1906.485|   Average|
|United States| North Carolina|-1862.3124|   Average|
|United States|       Virginia|    83.281|   Average|
|  Afghanistan|          Kabul|         0|   Average|
| Saudi Arabia|          Jizan|         0|   Average|
|       Brazil|         Parana|         0|   Average|
|        China|   Heilongjiang|         0|   Average|
|       France|  Ile-de-Fran

In [59]:
#any column if we want to rename

sales_US=sales_US.withColumnRenamed("totSales","TotalSales")

In [58]:
sales_US.drop("AvgSales")

DataFrame[Country: string, state: string, TotalSales: bigint]

In [61]:
#data cleansing
#dropna
#fillna

employeedf =spark.read.option("header","true").csv("/home/labuser/datasets/employee.csv") 

In [62]:
employeedf.dropna().show()

+-----+-----+----+
|empno|ename| sal|
+-----+-----+----+
|  111|  zzz|8000|
|  111|  aaa|8888|
|  121|  bbb|8000|
| NULL|  ccc|9000|
|false|  ddd|6000|
|  555|  eee|7000|
|  666|  fff|8890|
|    a|    b|   c|
|    x| NULL|NULL|
| 1000| null|null|
|  222| NULL|NULL|
+-----+-----+----+



In [63]:
employeedf.dropna("all").show()

+-----+-----+----+
|empno|ename| sal|
+-----+-----+----+
|  111|  zzz|8000|
|  111|  aaa|8888|
|  121|  bbb|8000|
| NULL|  ccc|9000|
|false|  ddd|6000|
|  555|  eee|7000|
|  765| null|null|
|  666|  fff|8890|
|  aaa| null|null|
| True| null|null|
| true| null|null|
|    a|    b|   c|
|    x| NULL|NULL|
| 1000| null|null|
|  222| NULL|NULL|
+-----+-----+----+



In [64]:
employeedf.dropna("any",2).show()

+-----+-----+----+
|empno|ename| sal|
+-----+-----+----+
|  111|  zzz|8000|
|  111|  aaa|8888|
|  121|  bbb|8000|
| NULL|  ccc|9000|
|false|  ddd|6000|
|  555|  eee|7000|
|  666|  fff|8890|
|    a|    b|   c|
|    x| NULL|NULL|
| 1000| null|null|
|  222| NULL|NULL|
+-----+-----+----+



In [65]:
from pyspark.sql.types import StructType,StructField, StringType,IntegerType,DoubleType

EmpSchema = StructType([  
    StructField('Empno', IntegerType(), True), 
    StructField('Empname', StringType(), True),    
     StructField('salary', DoubleType(), True) 
])

In [66]:
employeedf=spark.read.option("header","true").schema(EmpSchema).csv("/home/labuser/datasets/employee.csv")

In [67]:
employeedf.dropna().show()

+-----+-------+------+
|Empno|Empname|salary|
+-----+-------+------+
|  111|    zzz|8000.0|
|  111|    aaa|8888.0|
|  121|    bbb|8000.0|
|  555|    eee|7000.0|
|  666|    fff|8890.0|
+-----+-------+------+



23/09/25 06:07:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: empno, ename, sal
 Schema: Empno, Empname, salary
Expected: Empname but found: ename
CSV file: file:///home/labuser/datasets/employee.csv


In [70]:
employeedf.na.fill(0).show()

+-----+-------+------+
|Empno|Empname|salary|
+-----+-------+------+
|  111|    zzz|8000.0|
|  111|    aaa|8888.0|
|  121|    bbb|8000.0|
|    0|    ccc|9000.0|
|    0|    ddd|6000.0|
|  555|    eee|7000.0|
|  765|   null|   0.0|
|  666|    fff|8890.0|
|    0|   null|   0.0|
|    0|   null|   0.0|
|    0|   null|   0.0|
|    0|      b|   0.0|
|    0|   NULL|   0.0|
| 1000|   null|   0.0|
|  222|   NULL|   0.0|
+-----+-------+------+



23/09/25 06:13:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: empno, ename, sal
 Schema: Empno, Empname, salary
Expected: Empname but found: ename
CSV file: file:///home/labuser/datasets/employee.csv


In [71]:
employeedf.na.fill("unknown").na.fill(0).show()

+-----+-------+------+
|Empno|Empname|salary|
+-----+-------+------+
|  111|    zzz|8000.0|
|  111|    aaa|8888.0|
|  121|    bbb|8000.0|
|    0|    ccc|9000.0|
|    0|    ddd|6000.0|
|  555|    eee|7000.0|
|  765|unknown|   0.0|
|  666|    fff|8890.0|
|    0|unknown|   0.0|
|    0|unknown|   0.0|
|    0|unknown|   0.0|
|    0|      b|   0.0|
|    0|   NULL|   0.0|
| 1000|   null|   0.0|
|  222|   NULL|   0.0|
+-----+-------+------+



23/09/25 06:13:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: empno, ename, sal
 Schema: Empno, Empname, salary
Expected: Empname but found: ename
CSV file: file:///home/labuser/datasets/employee.csv


In [72]:
employeedf.na.fill({'salary':1000,'Empname':'unknown'}).dropna().show()

+-----+-------+------+
|Empno|Empname|salary|
+-----+-------+------+
|  111|    zzz|8000.0|
|  111|    aaa|8888.0|
|  121|    bbb|8000.0|
|  555|    eee|7000.0|
|  765|unknown|1000.0|
|  666|    fff|8890.0|
| 1000|   null|1000.0|
|  222|   NULL|1000.0|
+-----+-------+------+



23/09/25 06:18:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: empno, ename, sal
 Schema: Empno, Empname, salary
Expected: Empname but found: ename
CSV file: file:///home/labuser/datasets/employee.csv


In [74]:
employeedf.na.replace("NULL",None).na.fill({'salary':1000,'Empname':'unknown'}).dropna().show()

+-----+-------+------+
|Empno|Empname|salary|
+-----+-------+------+
|  111|    zzz|8000.0|
|  111|    aaa|8888.0|
|  121|    bbb|8000.0|
|  555|    eee|7000.0|
|  765|unknown|1000.0|
|  666|    fff|8890.0|
| 1000|   null|1000.0|
|  222|unknown|1000.0|
+-----+-------+------+



23/09/25 06:23:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: empno, ename, sal
 Schema: Empno, Empname, salary
Expected: Empname but found: ename
CSV file: file:///home/labuser/datasets/employee.csv


In [75]:
employeedf.na.replace(["NULL",None],["null",None],"Empname").na.fill({'salary':1000,'Empname':'unknown'}).dropna().show()

ValueError: Mixed type replacements are not supported